# Extração de texto do certificado Udemy

In [ ]:
!pip install pdf2image

In [ ]:
import cv2
from glob import glob
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import pytesseract
from typing import Any
from pdf2image import convert_from_path
from pdf2image.exceptions import PDFPageCountError

### Funções básicas 

In [ ]:
# Caminho
path = os.getcwd()

INFO_LIMPO = []
NOME = []
LOGCORROMPIDO = []

def verificar_pasta(caminho: str) -> Any:

  if os.path.isdir(caminho) == False:
      print(f'A pasta {caminho} não existe. Criando diretório.')
      os.mkdir(caminho)
  else:
      print(f'A pasta {caminho} existe.')


verificar_pasta(path + r'/pdf')

pdfs_files = glob(os.path.join(os.getcwd(), path + r'/pdf/*.pdf'))

In [ ]:
for fn in range(len(pdfs_files)):
    fn=0

    try:
        # Converter pdf para imagem
        images = convert_from_path(pdfs_files[fn])

    except PDFPageCountError as e:
        print('Nao foi possível ler o pdf. Corrompido')
        LOGCORROMPIDO.append(pdfs_files[fn])
        pass

    for i in range(len(images)):
        images[i].save('page.png', 'PNG')

    # Leitura da imagem
    img = cv2.imread('page.png', cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    try:

        # Encontrar as palavras na imagem
        boxes = pytesseract.image_to_data(img, lang='por')
        INFO = []  # Armazenar as informações obtidas da imagem-pdf

        # Encontrar boxes com as infos
        for a, b in enumerate(boxes.splitlines()):
            print(b)
            if a != 0:
                b = b.split()
                if len(b) == 12:
                    INFO.append(b)

        # Limpando as informações dos boxes
        INFO_LIMPO = []
        for i in range(len(INFO)):
            #print(INFO[i][-1])
            INFO_LIMPO.append(INFO[i][-1])

        INFO_LIMPO = ' '.join(INFO_LIMPO)
        print(INFO_LIMPO)

    except :
        print('Nao foi possível ler o pdf')

    # Nome
    try:
        # Nome do aluno
        NOME.append(re.search(r"certify that (.*[A-Za-z\s]) successfully", INFO_LIMPO).group(1))
    except AttributeError:
        print('Nome nao encontrado')

In [ ]:
df = pd.DataFrame({'Nome': NOME})
df